In [7]:
import pandas as pd
import numpy as np
import pickle
from collections import Counter

In [2]:
#pre = "D:\python\LEO_TM\BERT_chinese_LM_processing\src\\"  # for pycharm only
# define_dict_path = pre + "../data/TextTokenize_zh/company_dict.txt"
ori_news_path =  "../data/TextTokenize_zh/Foxconn_News_2018.csv"  # POC_NEWS.csv
tok_news_path = "../Results/tokenize_Foxconn2018_news_Leo.xlsx"
senta_path = "../data/TextTokenize_zh/Foxconn_Senta.txt"
three_algo_checkpoint =  "../data/TextTokenize_zh/Foxconn_3algo.pkl"
four_algo_checkpoint = "./Senta/SentaDL_output.pkl"
sentiment_result_path = "../Results/sentiment_Foxconn2018_news_Leo.xlsx"

In [3]:
three_algos = pickle.load(open(three_algo_checkpoint,'rb'))
fourth_algo = pickle.load(open(four_algo_checkpoint,'rb'))
four_algos = pd.merge(three_algos,fourth_algo,left_on='Unnamed: 0',right_on='input_id',how='outer')
four_algos = four_algos.filter(['input_id','tb_score','sn_score','bx_score','senta_score'])
four_algos

,input_id,tb_score,sn_score,bx_score,senta_score
0,0,-0.006981,-0.148375,0.45,-0.996373
1,1,0.037634,-0.491682,0.11,-0.994906
2,2,0.038169,-0.561931,0.87,0.967750
3,3,0.057605,-0.308602,0.55,-0.995725
4,4,0.000788,-0.839621,0.22,-0.997490
...,...,...,...,...,...
2623,2623,0.024561,-0.550450,0.10,-0.984371
2624,2624,-0.039489,-0.487939,0.62,-0.995011
2625,2625,0.050933,-0.746324,-0.10,-0.996006
2626,2626,0.020191,-0.587397,-0.05,-0.994787


In [4]:
four_algos['final_sentiment'] = (four_algos['tb_score']*2 + four_algos['sn_score'] + four_algos['bx_score']*2.3 + four_algos['senta_score']*0.8 )/6.1
four_algos1 = four_algos[four_algos.isna().any(axis=1)]
print(four_algos1)
col1 = four_algos.tb_score.mean()
col2 = four_algos.final_sentiment.mean()
naid_li = four_algos1.input_id.tolist()
for naid in naid_li:
    if four_algos.loc[naid].isna()['tb_score'] == True: #deal with possible error in TextBlob pkg (missing value)
        four_algos.loc[naid,'tb_score'] = ((four_algos.loc[naid,'sn_score'] + 2.1*four_algos.loc[naid,'bx_score'] + 0.9*four_algos.loc[naid,'senta_score'])/2 +col1+col2)/4
        four_algos.loc[naid,'final_sentiment'] = (four_algos.loc[naid,'tb_score']*2 + four_algos.loc[naid,'sn_score'] + four_algos.loc[naid,'bx_score']*2.3 + four_algos.loc[naid,'senta_score']*0.8 )/6.1
    else:
        print("fillna error, because the nan column is not TextBlob slot! => ",four_algos.loc[naid])
assert len(four_algos[four_algos.isna().any(axis=1)]) == 0
four_algos

      input_id  tb_score  sn_score  bx_score  senta_score  final_sentiment
216        216       NaN  0.455557      0.53    -0.969790              NaN
734        734       NaN  0.190694      0.00    -0.983106              NaN
1663      1663       NaN -0.196915      0.50    -0.984960              NaN
2454      2454       NaN  0.151552     -0.33    -0.982992              NaN


,input_id,tb_score,sn_score,bx_score,senta_score,final_sentiment
0,0,-0.006981,-0.148375,0.45,-0.996373,0.012388
1,1,0.037634,-0.491682,0.11,-0.994906,-0.157269
2,2,0.038169,-0.561931,0.87,0.967750,0.375345
3,3,0.057605,-0.308602,0.55,-0.995725,0.045087
4,4,0.000788,-0.839621,0.22,-0.997490,-0.185252
...,...,...,...,...,...,...
2623,2623,0.024561,-0.550450,0.10,-0.984371,-0.173578
2624,2624,-0.039489,-0.487939,0.62,-0.995011,0.010340
2625,2625,0.050933,-0.746324,-0.10,-0.996006,-0.273978
2626,2626,0.020191,-0.587397,-0.05,-0.994787,-0.238991


In [5]:
final_algo = four_algos.filter(['input_id','final_sentiment'])
tok_news_df = pd.read_excel(tok_news_path)
# tok_news_df.drop(['Unnamed: 0'],axis=1,inplace=True)
final_df = pd.merge(tok_news_df,final_algo,left_on='Unnamed: 0', right_on='input_id', how='outer')
final_df = final_df.drop(['Unnamed: 0','input_id'],axis=1)
# final_df = final_df.astype(str)
final_df

,ori_title,ori_news,tok_title_news,keyWord_algorithm,final_sentiment
0,鴻海號召百萬員工回鄉實現小老闆夢,新增網友看法：鴻海集團號召百萬員工回鄉創業，自己當小老闆，根據《經濟日報》報導，據了解，隨著...,鴻海 號召 百萬 員工 回 鄉 實現 小 老闆 夢 ： 新增 網友 看法 ： 鴻海 集團 號...,鴻海、員工、大陸,0.012388
1,熱門股—鴻海年末點燃反攻火種外資封關前連3買助威,一度遭遇外資連續 23 天賣超的鴻海 (2317-TW)，新曆年封關前夕在外資轉為連續 3 ...,熱門股 — 鴻海 年末 點燃 反攻 火種 外資 封關 前 連 3 買助威 ： 一度 遭遇 外...,鴻海、外資、封關、連續、賣超、股價,-0.157269
2,鴻海 尾牙總獎金額度保密,鴻海預定二月十一日舉辦尾牙，規模有望大幅超越去年。圖為鴻海董事長郭台銘。 圖／聯合報系資料照...,鴻海 尾牙 總獎金 額度 保密 ： 鴻海 預定 二月 十一日 舉辦 尾牙 規模 有望...,鴻海、尾牙、去年、今年,0.375345
3,鴻海號召百萬員工回鄉創業,鴻海集團近期積極投資新創之外，也鼓勵內部員工創新與創業。據了解，隨著傳統年節將至，鴻海富士康...,鴻海 號召 百萬 員工 回 鄉 創業 ： 鴻海 集團 近期 積極 投資 新創 之外 也 鼓...,鴻海、員工、創業、集團、大陸、富連網,0.045087
4,鴻海擬售夏普C類股估獲利663億早盤小漲,（中央社記者鍾榮峰台北2日電）鴻海早盤最高來到95.7元，小漲0.5%。鴻海擬處分夏普C類特...,鴻海 擬售 夏普 C 類股 估 獲利 663億 早盤 小漲 ： （ 中央社 記者 鍾榮峰 台...,鴻海、夏普、特別股、日圓,-0.185252
...,...,...,...,...,...
2623,路透：最快明年 鴻海首度在印度組裝高階iPhone,一名知情的消息人士透露，蘋果公司（Apple Inc.）最快2019年將透過代工業者鴻海集團...,路透 ： 最 快 明年 鴻海 首度 在 印度 組裝 高階 iPhone ： 一 名 知...,鴻海、印度、組裝、蘋果、投資、路透社、iphone,-0.173578
2624,借東風！郭台銘要靠夏普來實現鴻海集團的半導體夢...,夏普社長戴正吳正式宣佈將在2019年4月正式分拆夏普半導體業務，分拆的部門主要負責包括大型積...,借 東風 ！ 郭台銘 要 靠 夏普 來 實現 鴻海 集團 的 半導體 夢 . . . ： 夏...,夏普、鴻海、半導體、戴正吳,0.010340
2625,外資今年追捧友達、賣鴻海不回頭,台股107年回顧<BR> 台股迎接2019年，回顧今（2018）年的台股，外資...,外資 今年 追捧 友達 、 賣 鴻海 不 回頭 ： 台股 107年 回顧 <BR> ...,外資、今年、鴻海、賣超、台積電、跌幅,-0.273978
2626,鴻海封關市值不到兆元 外資今年累積大賣千億,▲鴻海集團董事長郭台銘今年6月與美國總統川普參加威斯康辛州工廠預定地破土儀式。（圖／達志影像...,鴻海 封關 市值 不到 兆元 外資 今年 累積 大賣 千億 ： 鴻海 集團 董事長 郭台...,鴻海、今年,-0.238991


In [6]:
def convert_level_OLD(s):
    """
    input: series (float number)
    output: convert to level
    """
    if 0.1>=s>=-0.1:
        return 'NEUTRAL'
    elif 0.3>=s>0.1:
        return "Positive (20%)"
    elif 0.5>=s>0.3:
        return "Positive (40%)"
    elif 0.7>=s>0.5:
        return "Positive (60%)"
    elif 0.9>=s>0.7:
        return "Positive (80%)"
    elif s>0.9:
        return "Positve (100%)"
    elif -0.3<=s<-0.1:
        return "Negative (20%)"
    elif -0.5<=s<0.3:
        return "Negative (40%)"
    elif -0.7<=s<-0.5:
        return "Negative (60%)"
    elif -0.9<=s<-0.7:
        return "Negative (80%)"
    elif s<-0.9:
        return "Negative (100%)"    

def convert_level(s):
    """
    input: series (float number)
    output: convert to level
    """
    if 0.2>s>-0.2:
        return 'NEUTRAL'
    elif 0.6>s>=0.2:
        return "Positive (40%)"
    elif s>=0.6:
        return "Positive (80%)"
    elif -0.6<s<=-0.2:
        return "Negative (40%)"
    elif s<=-0.6:
        return "Negative (80%)"
    else : #exception
        return str(s)      
    
final_df['sentiment_level'] = final_df.final_sentiment.map(convert_level)
final_df.to_excel(sentiment_result_path)
final_df

,ori_title,ori_news,tok_title_news,keyWord_algorithm,final_sentiment,sentiment_level
0,鴻海號召百萬員工回鄉實現小老闆夢,新增網友看法：鴻海集團號召百萬員工回鄉創業，自己當小老闆，根據《經濟日報》報導，據了解，隨著...,鴻海 號召 百萬 員工 回 鄉 實現 小 老闆 夢 ： 新增 網友 看法 ： 鴻海 集團 號...,鴻海、員工、大陸,0.012388,NEUTRAL
1,熱門股—鴻海年末點燃反攻火種外資封關前連3買助威,一度遭遇外資連續 23 天賣超的鴻海 (2317-TW)，新曆年封關前夕在外資轉為連續 3 ...,熱門股 — 鴻海 年末 點燃 反攻 火種 外資 封關 前 連 3 買助威 ： 一度 遭遇 外...,鴻海、外資、封關、連續、賣超、股價,-0.157269,NEUTRAL
2,鴻海 尾牙總獎金額度保密,鴻海預定二月十一日舉辦尾牙，規模有望大幅超越去年。圖為鴻海董事長郭台銘。 圖／聯合報系資料照...,鴻海 尾牙 總獎金 額度 保密 ： 鴻海 預定 二月 十一日 舉辦 尾牙 規模 有望...,鴻海、尾牙、去年、今年,0.375345,Positive (40%)
3,鴻海號召百萬員工回鄉創業,鴻海集團近期積極投資新創之外，也鼓勵內部員工創新與創業。據了解，隨著傳統年節將至，鴻海富士康...,鴻海 號召 百萬 員工 回 鄉 創業 ： 鴻海 集團 近期 積極 投資 新創 之外 也 鼓...,鴻海、員工、創業、集團、大陸、富連網,0.045087,NEUTRAL
4,鴻海擬售夏普C類股估獲利663億早盤小漲,（中央社記者鍾榮峰台北2日電）鴻海早盤最高來到95.7元，小漲0.5%。鴻海擬處分夏普C類特...,鴻海 擬售 夏普 C 類股 估 獲利 663億 早盤 小漲 ： （ 中央社 記者 鍾榮峰 台...,鴻海、夏普、特別股、日圓,-0.185252,NEUTRAL
...,...,...,...,...,...,...
2623,路透：最快明年 鴻海首度在印度組裝高階iPhone,一名知情的消息人士透露，蘋果公司（Apple Inc.）最快2019年將透過代工業者鴻海集團...,路透 ： 最 快 明年 鴻海 首度 在 印度 組裝 高階 iPhone ： 一 名 知...,鴻海、印度、組裝、蘋果、投資、路透社、iphone,-0.173578,NEUTRAL
2624,借東風！郭台銘要靠夏普來實現鴻海集團的半導體夢...,夏普社長戴正吳正式宣佈將在2019年4月正式分拆夏普半導體業務，分拆的部門主要負責包括大型積...,借 東風 ！ 郭台銘 要 靠 夏普 來 實現 鴻海 集團 的 半導體 夢 . . . ： 夏...,夏普、鴻海、半導體、戴正吳,0.010340,NEUTRAL
2625,外資今年追捧友達、賣鴻海不回頭,台股107年回顧<BR> 台股迎接2019年，回顧今（2018）年的台股，外資...,外資 今年 追捧 友達 、 賣 鴻海 不 回頭 ： 台股 107年 回顧 <BR> ...,外資、今年、鴻海、賣超、台積電、跌幅,-0.273978,Negative (40%)
2626,鴻海封關市值不到兆元 外資今年累積大賣千億,▲鴻海集團董事長郭台銘今年6月與美國總統川普參加威斯康辛州工廠預定地破土儀式。（圖／達志影像...,鴻海 封關 市值 不到 兆元 外資 今年 累積 大賣 千億 ： 鴻海 集團 董事長 郭台...,鴻海、今年,-0.238991,Negative (40%)


In [10]:
test_stat = final_df['sentiment_level'].tolist()
print(dict(Counter(test_stat)))

{'NEUTRAL': 1931, 'Positive (40%)': 244, 'Negative (40%)': 451, 'Negative (80%)': 2}
